# Binary Classification Deep Learning Model for Cats and Dogs Using Keras Take 8
### David Lowe
### February 4, 2020

Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: The purpose of this project is to construct a predictive model using various machine learning algorithms and to document the end-to-end steps using a template. The "Cats and Dogs" dataset is a binary classification situation where we are trying to predict one of the two possible outcomes.

INTRODUCTION: Web services are often protected with a challenge that's supposed to be easy for people to solve, but difficult for computers. Such a challenge is often called a CAPTCHA (Completely Automated Public Turing test to tell Computers and Humans Apart) or HIP (Human Interactive Proof). ASIRRA (Animal Species Image Recognition for Restricting Access) is a HIP that works by asking users to identify photographs of cats and dogs. This task is difficult for computers, but studies have shown that people can accomplish it quickly and accurately.

The current literature suggests that machine classifiers can score above 80% accuracy on this task. Therefore, ASIRRA is no longer considered safe from attack. Kaggle created a contest to benchmark the latest computer vision and deep learning approaches to this problem. The training archive contains 25,000 images of dogs and cats. We will need to train our algorithm on these files and predict the correct labels for the test dataset.

In iteration Take1, we constructed a simple VGG convolutional model with 1 VGG block to classify the images. This model serves as the baseline for the future iterations of modeling.

In iteration Take2, we constructed a simple VGG convolutional model with 2 VGG blocks to classify the images. The additional modeling enabled us to improve our baseline model.

In iteration Take3, we constructed a simple VGG convolutional model with 3 VGG blocks to classify the images. The additional modeling enabled us to improve our baseline model further.

In iteration Take4, we applied dropout to our VGG-3 model. The addition of the dropout layers improved our model.

In iteration Take5, we applied image data augmentation to our VGG-3 model. The addition of the image data augmentation improved our model.

In iteration Take6, we applied both dropout layers and image data augmentation to our VGG-3 model. The addition of both techniques improved our model but not much more from the previous VGG-3 models.

In iteration Take7, we applied the transfer learning technique by incorporating the VGG-16 model into our classifier. The incorporation of the VGG-16 model brought our model's performance closer to the Kaggle results.

In this iteration, we will take the TensorFlow/Keras model generated and saved from Take7 to make predictions on brand new images. We will upload the prediction to Kaggle and receive an accuracy score for our predictions.

ANALYSIS: In iteration Take1, the performance of the Take1 model achieved an accuracy score of 95.55% after training for 20 epochs. The same model, however, processed the test dataset with an accuracy of only 72.99% after 20 epochs. Reviewing the plot, we can see that the model was starting to overfit the training dataset after only ten epochs. We will need to explore other modeling approaches to reduce the over-fitting.

In iteration Take2, the performance of the Take2 model achieved an accuracy score of 97.94% after training for 20 epochs. The same model, however, processed the test dataset with an accuracy of only 75.67% after 20 epochs. Reviewing the plot, we can see that the model was starting to overfit the training dataset after only seven epochs. We will need to explore other modeling approaches to reduce the over-fitting.

In iteration Take3, the performance of the Take3 model achieved an accuracy score of 97.14% after training for 20 epochs. The same model, however, processed the test dataset with an accuracy of only 80.19% after 20 epochs. Reviewing the plot, we can see that the model was starting to overfit the training dataset after only six epochs. We will need to explore other modeling approaches to reduce the over-fitting.

In iteration Take4, the performance of the Take4 model achieved an accuracy score of 86.92% after training for 50 epochs. The same model, however, processed the test dataset with an accuracy of 81.04% after 50 epochs. By reviewing the plot, this iteration indicated to us that having dropout layers can be a good tactic to improve the model's predictive performance.

In iteration Take5, the performance of the Take5 model achieved an accuracy score of 87.52% after training for 50 epochs. The same model, however, processed the test dataset with an accuracy of 85.12% after 50 epochs. By reviewing the plot, this iteration indicated to us that having image data augmentation can be a good tactic to improve the model's predictive performance.

In iteration Take6, the performance of the Take6 model achieved an accuracy score of 88.60% after training for 200 epochs. The same model, however, processed the test dataset with an accuracy of 87.25% after 200 epochs. By reviewing the plot, this iteration indicated to us that having both dropout layers and image data augmentation can create a low-variance model that does not overfit too early in the modeling process.

In iteration Take7, the performance of the Take7 model achieved an accuracy score of 99.99% after training for 10 epochs. The same model processed the test dataset with an accuracy of 97.77% after 10 epochs. This iteration indicated to us that a workable model will requie more layers and depth than having simply three VGG blocks.

In this iteration, the VGG-16/Take7 model generated a submission file that can be submitted to Kaggle for further evaluation.

CONCLUSION: For this dataset, the model built using Keras and TensorFlow achieved a comparable result with the Kaggle competition. We can use this model to predict other previously unseen/processed pet images.

Dataset Used: Cats and Dogs Dataset

Dataset ML Model: Binary classification with numerical attributes

Dataset Reference: https://www.microsoft.com/en-us/download/details.aspx?id=54765

One potential source of performance benchmarks: https://www.kaggle.com/c/dogs-vs-cats/overview

# Section 0. Prepare Environment

In [39]:
# Load libraries and packages
import pandas as pd
import os
import sys
import shutil
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [40]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the verbose and debug flags to print detailed messages for debugging (setting True will activate!)
verbose = False
debug = False

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

In [41]:
# Set up the email notification function
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Keras Binary Classification Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [42]:
if (notifyStatus): email_notify("Phase 0 Prepare Environment completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 1. Load Image

In [43]:
if (notifyStatus): email_notify("Phase 1 Load Image has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [44]:
# Data Pre-Processing Tasks to perform at this time
# Extract/Unzip the Kaggle testing images Zip file (test1.zip) into a directory (e.g. test1/)
# Copy the final model file (final_model_cats_and_dogs.h5) to the script subdirectory

In [45]:
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img

In [46]:
if (notifyStatus): email_notify("Phase 1 Load Image completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 2. Load Model

In [47]:
if (notifyStatus): email_notify("Phase 2 Load Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [48]:
# Load the Keras model for making predictions
production_model = load_model('final_model_cats_and_dogs.h5')

In [49]:
if (notifyStatus): email_notify("Phase 2 Load Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 3. Make Precitions

In [58]:
if (notifyStatus): email_notify("Phase 3 Make Predictions has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [59]:
# Set up the dataframe with the prediction results
submission_df = pd.DataFrame(columns=['id','label'])
i = 0

# load an image and predict the class
src_directory = 'test1/'
for file_name in os.listdir(src_directory):
    file_id = int(os.path.splitext(file_name)[0])
    if (verbose): print('Processing image', file_name, 'as file ID:', file_id)
    # load the image
    img_file = src_directory + file_name
    img = load_image(img_file)
    if (verbose): print('Finished pre-processing of image', img_file)
    # predict the class
    result = production_model.predict(img)
    pet_label = int(result[0])
    submission_df.loc[i] = [file_id, pet_label]
    i = i + 1
    if ((i % 500)==0): print('Number of image processed so far:', i)
print('Total number of image processed:', i)

Number of image processed so far: 500
Number of image processed so far: 1000
Number of image processed so far: 1500
Number of image processed so far: 2000
Number of image processed so far: 2500
Number of image processed so far: 3000
Number of image processed so far: 3500
Number of image processed so far: 4000
Number of image processed so far: 4500
Number of image processed so far: 5000
Number of image processed so far: 5500
Number of image processed so far: 6000
Number of image processed so far: 6500
Number of image processed so far: 7000
Number of image processed so far: 7500
Number of image processed so far: 8000
Number of image processed so far: 8500
Number of image processed so far: 9000
Number of image processed so far: 9500
Number of image processed so far: 10000
Number of image processed so far: 10500
Number of image processed so far: 11000
Number of image processed so far: 11500
Number of image processed so far: 12000
Number of image processed so far: 12500
Total number of imag

In [60]:
if (notifyStatus): email_notify("Phase 3 Make Predictions completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 4. Produce Output

In [61]:
if (notifyStatus): email_notify("Phase 4 Produce Output has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [62]:
# Perform a quick check of the dataframe content
print(submission_df.head(20))

       id label
0       1     1
1      10     0
2     100     0
3    1000     1
4   10000     1
5   10001     0
6   10002     0
7   10003     1
8   10004     1
9   10005     0
10  10006     0
11  10007     0
12  10008     0
13  10009     0
14   1001     0
15  10010     1
16  10011     1
17  10012     0
18  10013     0
19  10014     0


In [63]:
# Not required for this iteration of the project
out_file = submission_df.to_csv(header=True, index=False)
filename = 'submission_' + datetime.now().strftime('%Y%m%d-%H%M') + '.csv'
with open(filename, 'w') as f:
    f.write(out_file)

In [64]:
if (notifyStatus): email_notify("Phase 4 Produce Output completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [65]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 1:09:22.235330
